In [8]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import yfinance as yf
from splinter import Browser
from bs4 import BeautifulSoup
import re
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://stockanalysis.com/stocks/'
browser.visit(url)
element = browser.find_by_name('perpage').first
element.select('10000')

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [C:\Users\bmontague\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\bmontague\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [10]:
html = browser.html   
soup = BeautifulSoup(html,'html.parser')

table = soup.find('table', {'class' : 'symbol-table index'})

symbol = []
company_name = []
industry = []
market_cap= []
for row in table.find_all('tr')[1:]:
    symbol.append(row.find_all('td')[0].text)
    company_name.append(row.find_all('td')[1].text)
    industry.append(row.find_all('td')[2].text)
    market_cap.append(row.find_all('td')[3].text)

browser.quit()

stocks_df = pd.DataFrame(list(zip(symbol, company_name,industry,market_cap)),
               columns =['Symbol', 'Company','Industry','Market Cap'])
stocks_df.to_csv('Output/Stocks/stock_list.csv',index=False)
stocks_df.head()




,Symbol,Company,Industry,Market Cap
0,A,Agilent Technologies,Life Sciences Tools & Services,49.09B
1,AA,Alcoa,Metals & Mining,8.92B
2,AAC,Ares Acquisition,Blank Check / SPAC,1.23B
3,AACG,ATA Creativity Global,Diversified Consumer Services,60.87M
4,AADI,Aadi Bioscience,Biotechnology,490.59M


In [ ]:
industry = pd.unique(industry)
x = 1
obj = {}
for i in industry:
    obj_list = f"l{x}"
    df = stocks_df.loc[stocks_df['Industry']==i]
    obj[obj_list] = df["Symbol"].to_list()
    x = x + 1

In [ ]:
x = 1
for i in industry:
    key = f"l{x}"
    obj[i]=obj.pop(key)
    x = x + 1   

In [ ]:
start = dt.datetime(2011, 1, 1)
end = dt.datetime.now() 

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in symbol:  
    
    # print the symbol which is being downloaded
    print( str(symbol.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
       None
stock_final["Symbol"]=stock_final["Name"]
stock_final.head()

In [ ]:
stock_final = pd.merge(stock_final,stocks_df, on = '')
stock_final.head()

In [ ]:
start = dt.datetime(2011, 1, 1)
end = dt.datetime.now()

for value in obj.values():
    industry = [k for k, v in obj.items() if v == value]
    industry = str([re.sub('[^a-zA-Z0-9]+', '', _) for _ in industry])
    size = len(industry)
    industry = industry[2:size-2]
    
    data = yf.download(value, start=start, end=end)
    data_df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
    data_df = pd.merge(data_df,stocks_df, on = "Symbol")
    data_df.to_csv(f'Output/{industry}.csv',index=False)
